# <span style="color:#6FFFE9">Push and get DF between python and MySQL</span> 

## <span style="color:#ffadad">Import libraries</span>

In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import sqlalchemy
from datetime import datetime, date, timedelta
from pytz import timezone
import api_pw
import json

## <span style="color:#ffadad">Get City Data </span>

In [43]:
# begin a for loop to create a dictionary of information for each city
cities = ['Berlin', 'Heidelberg', 'Cologne', 'Taipei', 'Munich', 'Kyoto']

# empty list that will be filled with one dictionary of information per city
list_for_df = []

for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    
    # Not for all cities the info for the population is on the same spot
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    
    
    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)

# make the DataFrame
cities_df = pd.DataFrame(list_for_df)

# fixing latitude
cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('N', '00', regex=False)
# fixing longitude
cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('E', '00', regex=False)

In [44]:
cities_df

,city,country,population,latitude,longitude
0,Berlin,Germany,"3,677,472",52.3112,13.2418
1,Heidelberg,Germany,"159,245",49.2500,08.4300
2,Cologne,Germany,"1,073,096",50.5611,6.5710
3,Taipei,Republic of China (Taiwan),"2,488,043",25.0215,121.3345
4,Munich,Germany,"1,487,708",48.0815,11.3430
5,Kyoto,Japan,"1,463,723",35.042,135.466


In [45]:
cities_df.iloc[3,1] = cities_df.iloc[3,1][-7:-1]
cities_df


,city,country,population,latitude,longitude
0,Berlin,Germany,"3,677,472",52.3112,13.2418
1,Heidelberg,Germany,"159,245",49.2500,08.4300
2,Cologne,Germany,"1,073,096",50.5611,6.5710
3,Taipei,Taiwan,"2,488,043",25.0215,121.3345
4,Munich,Germany,"1,487,708",48.0815,11.3430
5,Kyoto,Japan,"1,463,723",35.042,135.466


In [46]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        6 non-null      object
 1   country     6 non-null      object
 2   population  6 non-null      object
 3   latitude    6 non-null      object
 4   longitude   6 non-null      object
dtypes: object(5)
memory usage: 368.0+ bytes


In [47]:
cities_df['population'] = cities_df['population'].str.replace(',', '', regex=False)
cities_df['population'] = cities_df['population'].astype(int)
cities_df['latitude'] = cities_df['latitude'].astype(float)
cities_df['longitude'] = cities_df['longitude'].astype(float)
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        6 non-null      object 
 1   country     6 non-null      object 
 2   population  6 non-null      int64  
 3   latitude    6 non-null      float64
 4   longitude   6 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 368.0+ bytes


## <span style="color:#ffadad">Push City data to MySQL </span>

In [2]:
schema="gans_local"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="api_pw.sql_pw" # MySQL Workbench password!!!! save it somewhere safe
port=3306
connection_details = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [48]:
cities_df.to_sql('city',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=connection_details,            # con-> connection string;
              index=False) 

6

## <span style="color:#ffadad">Get the City table from MySQL </span>

In [3]:
city_df = pd.read_sql('city',con=connection_details)
city_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3677472,52.3112,13.2418
1,2,Heidelberg,Germany,159245,49.2500,8.4300
2,3,Cologne,Germany,1073096,50.5611,6.5710
3,4,Taipei,Taiwan,2488043,25.0215,121.3350
4,5,Munich,Germany,1487708,48.0815,11.3430
5,6,Kyoto,Japan,1463723,35.0420,135.4660


## <span style="color:#ffadad">Preparing Weather DataFrame </span>

In [6]:
def get_weather(city_df):

    api_key = api_pw.openweather_api

    #create a dictionary weather_cities_dict
    weather_cities_dict = {"city_id"         : [],
                           "Forecast_time"   : [],
                           "Weather_desc"    : [],
                           "Temperature"     : [],
                           "Rain_probability": [],
                           "Humidity"        : []
                          }

    for i, city in enumerate(city_df['city_id']):
        weather_cities = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?lat={city_df.iloc[i]['latitude']}&lon={city_df.iloc[i]['longitude']}&appid={api_key}&units=metric").json()

        for j in weather_cities["list"]:
            weather_cities_dict["city_id"].append(city)
            weather_cities_dict['Forecast_time'].append(j['dt_txt'])
            weather_cities_dict['Weather_desc'].append(j['weather'][0]['description'])
            weather_cities_dict['Temperature'].append(j['main']['temp'])
            weather_cities_dict['Humidity'].append(j['main']['humidity'])

            try:
                weather_cities_dict['Rain_probability'].append(j['rain']['3h'])
            except:
                weather_cities_dict['Rain_probability'].append('0')


    weather_df = pd.DataFrame(weather_cities_dict)

    return weather_df

In [8]:
weather_df = get_weather(city_df)
weather_df

,city_id,Forecast_time,Weather_desc,Temperature,Rain_probability,Humidity
0,1,2023-04-04 15:00:00,overcast clouds,7.53,0,43
1,1,2023-04-04 18:00:00,overcast clouds,6.21,0,51
2,1,2023-04-04 21:00:00,overcast clouds,3.26,0,62
3,1,2023-04-05 00:00:00,broken clouds,0.05,0,76
4,1,2023-04-05 03:00:00,scattered clouds,-0.53,0,81
...,...,...,...,...,...,...
235,6,2023-04-09 00:00:00,clear sky,9.88,0,53
236,6,2023-04-09 03:00:00,clear sky,14.88,0,35
237,6,2023-04-09 06:00:00,clear sky,15.54,0,32
238,6,2023-04-09 09:00:00,clear sky,10.75,0,54


In [55]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City              240 non-null    object 
 1   Country           240 non-null    object 
 2   Forecast_time     240 non-null    object 
 3   Weather_desc      240 non-null    object 
 4   Temperature       240 non-null    float64
 5   Rain_probability  240 non-null    object 
 6   Humidity          240 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 13.2+ KB


## <span style="color:#ffadad">Push weather table to MySQL </span>

In [9]:
weather_df.to_sql('weather',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=connection_details,            # con-> connection string;
              index=False) 

240

## <span style="color:#ffadad">Get the icao table from MySQL </span>

In [3]:
icao_df = pd.read_sql('icao',con=connection_details)
icao_df

,city_id,icao,airport_name
0,1,EDDB,Berlin Brandenburg Airport
1,3,EDDK,Cologne Bonn Airport
2,5,EDDM,Munich International Airport
3,2,EDIU,Heidelberg Airport
4,4,RCTP,Taoyuan International Airport
5,6,RJBB,Kansai International Airport


In [7]:
icao_df["icao"]

0    EDDB
1    EDDK
2    EDDM
3    EDIU
4    RCTP
5    RJBB
Name: icao, dtype: object

In [31]:
icao_df1 = icao_df.loc[icao_df["icao"].isin(["EDDB", "RCTP"])]
icao_df1

,city_id,icao,airport_name
0,1,EDDB,Berlin Brandenburg Airport
4,4,RCTP,Taoyuan International Airport


In [32]:
for i, icao in enumerate(icao_df1["icao"]):
    print( icao)

EDDB
RCTP


## <span style="color:#ffadad">Preparing Flight DataFrame </span>

In [33]:
def tomorrows_flight_arrivals(icao_df):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  # we need the data from tomorrow, we need to set a baseline for today
  # so that can use timedelta to get the time for tomorrow
  tomorrow = (today + timedelta(days=1))

  list_for_df = [] # prepare a empty list for the df

  for i, icao in enumerate(icao_df["icao"]):
    times = [["00:00","11:59"],["12:00","23:59"]] # how to get the 24h data

    for time in times: # doing time loop for 2*12H
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
      headers = {
	      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        "X-RapidAPI-Key": "api_pw.flight_api"
                }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)        
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  flight_df = pd.DataFrame(list_for_df)
  flight_df["arrival_time_local"] = pd.to_datetime(flight_df["arrival_time_local"])
  flight_df["departure_time_local"] = pd.to_datetime(flight_df["departure_time_local"],utc=True)
  flight_df["data_retrieved_on"] = pd.to_datetime(flight_df["data_retrieved_on"])
  flight_df["arrival_terminal"] = flight_df["arrival_terminal"].astype("int")
  return flight_df



In [34]:
flight_df = tomorrows_flight_arrivals(icao_df1)
flight_df

,arrival_icao,flight_number,airline,arrival_terminal,arrival_time_local,departure_icao,departure_city,departure_time_local,data_retrieved_on
0,EDDB,XQ 966,SunExpress,1,2023-04-07 05:45:00+02:00,LTBJ,İzmir,2023-04-07 00:40:00+00:00,2023-04-06
1,EDDB,HU 489,Hainan,1,2023-04-07 06:05:00+02:00,ZBAA,Beijing,2023-04-06 18:00:00+00:00,2023-04-06
2,EDDB,QR 79,Qatar Airways,1,2023-04-07 06:55:00+02:00,OTHH,Doha,2023-04-06 22:50:00+00:00,2023-04-06
3,EDDB,W6 4515,Wizz Air,0,2023-04-07 06:40:00+02:00,LBWN,Varna,2023-04-07 02:10:00+00:00,2023-04-06
4,EDDB,LH 170,Lufthansa,1,2023-04-07 07:55:00+02:00,EDDF,Frankfurt-am-Main,2023-04-07 04:45:00+00:00,2023-04-06
...,...,...,...,...,...,...,...,...,...
436,RCTP,JX 742,Starlux,1,2023-04-07 22:40:00+08:00,VTBS,Bangkok,2023-04-07 10:45:00+00:00,2023-04-06
437,RCTP,7C 2653,Jeju Air,1,2023-04-07 23:30:00+08:00,RKPK,Busan-si,2023-04-07 12:55:00+00:00,2023-04-06
438,RCTP,BR 195,EVA Air,2,2023-04-07 23:20:00+08:00,RJAA,Tokyo,2023-04-07 11:40:00+00:00,2023-04-06
439,RCTP,BR 715,EVA Air,2,2023-04-07 23:55:00+08:00,ZBAA,Beijing,2023-04-07 12:40:00+00:00,2023-04-06


## <span style="color:#ffadad">Push flight table to MySQL </span>

In [21]:
flight_df.to_sql('flight',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=connection_details,            # con-> connection string;
              index=False) 

213